In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

print("=" * 80)
print("CREATING FLOOD MASTER CSV FROM RAW DATA")
print("=" * 80)

# ============================================================================
# STEP 0: FIGURE OUT THE CORRECT PATHS
# ============================================================================

print("\n" + "="*80)
print("STEP 0: DETERMINING CORRECT PATHS")
print("="*80)

current_dir = os.getcwd()
print(f"\nCurrent working directory: {current_dir}")

# Check if we're in notebooks folder
if 'notebooks' in current_dir:
    print("✓ Running from notebooks folder, adjusting paths...")
    base_path = '../'
else:
    print("✓ Running from project root")
    base_path = './'

# ============================================================================
# STEP 1: FIND RAW DATA FILES
# ============================================================================

print("\n" + "="*80)
print("STEP 1: DISCOVERING RAW DATA FILES")
print("="*80)

# Try multiple possible paths
possible_raw_paths = [
    os.path.join(base_path, 'data/raw/flood'),
    os.path.join(base_path, 'data/raw'),
    os.path.join(base_path, 'data'),
    'data/raw/flood',
    'data/raw',
    'data',
]

print("\nChecking for raw flood data directory...")
raw_data_path = None

for path in possible_raw_paths:
    if os.path.exists(path):
        print(f"✓ Found: {path}")
        raw_data_path = path
        break

if raw_data_path is None:
    print("❌ Could not find data directory!")
    print("\nTrying to find CSV files anywhere...")
    
    import glob
    all_csvs = glob.glob('**/*.csv', recursive=True)
    
    if all_csvs:
        print(f"\nFound {len(all_csvs)} CSV files total:")
        for csv in all_csvs[:15]:
            print(f"  • {csv}")
    else:
        print("❌ No CSV files found!")
        print("\nPlease check your directory structure and update file paths manually.")
        exit()
else:
    print(f"\n✓ Data directory: {raw_data_path}")
    files = os.listdir(raw_data_path)
    print(f"  Contains {len(files)} files/folders:")
    for item in files:
        full_path = os.path.join(raw_data_path, item)
        if os.path.isfile(full_path):
            size = os.path.getsize(full_path) / 1024
            print(f"    • {item} ({size:.0f} KB)")
        else:
            print(f"    📁 {item}/")

# ============================================================================
# STEP 2: MANUALLY LOAD CSVs (BASED ON YOUR SCREENSHOTS)
# ============================================================================

print("\n" + "="*80)
print("STEP 2: LOADING RAW DATA FILES")
print("="*80)

# Based on your screenshots, you have these files
# Adjust these paths if needed!
csv_files_to_try = [
    'DFSI.csv',
    'District_FloodImpact.csv',
    'India_Flood_Inventory_v3.csv',
    'DFO_2507_From_20040620_to_20041007.csv',
    'DFO_4673_From_20180901_to_20180907.csv',
    'Indian_earthquake_data.csv',
    'cyclones-1891-2016.csv',
]

loaded_dfs = {}

for filename in csv_files_to_try:
    # Try to find this file in raw data path
    if raw_data_path:
        file_path = os.path.join(raw_data_path, filename)
        
        # Try flood subdirectory
        if not os.path.exists(file_path):
            file_path = os.path.join(raw_data_path, 'flood', filename)
        
        # Try without flood subdirectory
        if not os.path.exists(file_path):
            file_path = os.path.join(raw_data_path.replace('/flood', ''), filename)
        
        # Try just in data folder
        if not os.path.exists(file_path):
            file_path = os.path.join(base_path, 'data', filename)
    else:
        file_path = filename
    
    if os.path.exists(file_path):
        print(f"\n✓ Loading: {filename}")
        try:
            df = pd.read_csv(file_path)
            loaded_dfs[filename] = df
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)[:5]}...")
        except Exception as e:
            print(f"  ❌ Error: {e}")
    # Don't print error if file not found (too much noise)

print(f"\n✓ Successfully loaded {len(loaded_dfs)} files")

# ============================================================================
# STEP 3: COMBINE LOADED DATA
# ============================================================================

if len(loaded_dfs) == 0:
    print("\n" + "="*80)
    print("⚠️ NO CSV FILES FOUND")
    print("="*80)
    print("""
SOLUTION:
1. Tell me your actual file paths
2. OR run this code to auto-find them:

import glob
import os
all_csvs = glob.glob('**/*.csv', recursive=True)
print("CSV files found:")
for csv in all_csvs:
    print(f"  {csv}")
""")
    exit()

print("\n" + "="*80)
print("STEP 3: COMBINING DATA")
print("="*80)

# Start with first file
master_df = list(loaded_dfs.values())[0].copy()
print(f"\nStarting with: {list(loaded_dfs.keys())[0]}")
print(f"  Shape: {master_df.shape}")

# Add other files
for filename, df in list(loaded_dfs.items())[1:]:
    print(f"\nAdding: {filename}")
    print(f"  Before: {master_df.shape}")
    
    # Just concatenate (you can improve merge logic later)
    master_df = pd.concat([master_df, df], ignore_index=True, axis=0)
    
    print(f"  After: {master_df.shape}")

# ============================================================================
# STEP 4: CLEAN DATA
# ============================================================================

print("\n" + "="*80)
print("STEP 4: CLEANING DATA")
print("="*80)

print(f"\nBefore: {master_df.shape}")

# Remove complete duplicates
master_df = master_df.drop_duplicates()
print(f"✓ Removed duplicates: {master_df.shape}")

# Remove rows that are completely empty
master_df = master_df.dropna(how='all')
print(f"✓ Removed empty rows: {master_df.shape}")

# Fill numeric columns with median
for col in master_df.select_dtypes(include=[np.number]).columns:
    if master_df[col].isnull().sum() > 0:
        master_df[col].fillna(master_df[col].median(), inplace=True)

print(f"✓ Filled missing numeric values")

# ============================================================================
# STEP 5: SAVE MASTER FILE
# ============================================================================

print("\n" + "="*80)
print("STEP 5: SAVING MASTER FILE")
print("="*80)

# Create processed directory
processed_dir = os.path.join(base_path, 'data/processed')
os.makedirs(processed_dir, exist_ok=True)

# Save master file
output_file = os.path.join(processed_dir, 'flood_master.csv')
master_df.to_csv(output_file, index=False)

print(f"\n✓ Saved: {output_file}")
print(f"  Rows: {len(master_df)}")
print(f"  Columns: {len(master_df.columns)}")
print(f"  Size: {os.path.getsize(output_file) / (1024**2):.2f} MB")

# ============================================================================
# STEP 6: AUDIT
# ============================================================================

print("\n" + "="*80)
print("STEP 6: FILE AUDIT")
print("="*80)

print(f"\nColumns:")
for i, col in enumerate(master_df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nFirst row:")
print(master_df.iloc[0])

print("\n" + "="*80)
print("✓ SUCCESS! flood_master.csv created")
print("="*80)

CREATING FLOOD MASTER CSV FROM RAW DATA

STEP 0: DETERMINING CORRECT PATHS

Current working directory: c:\SEM III\GBM_sih\notebooks
✓ Running from notebooks folder, adjusting paths...

STEP 1: DISCOVERING RAW DATA FILES

Checking for raw flood data directory...
✓ Found: ../data/raw/flood

✓ Data directory: ../data/raw/flood
  Contains 10 files/folders:
    📁 archive/
    • archive.zip (845 KB)
    📁 DFO_2507_From_20040620_to_20041007/
    • DFO_2507_From_20040620_to_20041007.zip (81020 KB)
    📁 DFO_4673_From_20180901_to_20180907/
    • DFO_4673_From_20180901_to_20180907.zip (15753 KB)
    • DFSI.csv (28 KB)
    • District_FloodedArea.csv (32 KB)
    • District_FloodImpact.csv (19 KB)
    • India_Flood_Inventory_v3.csv (1759 KB)

STEP 2: LOADING RAW DATA FILES

✓ Loading: DFSI.csv
  Shape: (744, 3)
  Columns: ['Unnamed: 0', 'State_Name', 'DFSI']...

✓ Loading: District_FloodImpact.csv
  Shape: (732, 5)
  Columns: ['Dist_Name', 'Human_fatality', 'Human_injured', 'Population', 'Mean_Floo

C:\Users\hp\AppData\Local\Temp\ipykernel_1692\1060689562.py:199: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df[col].fillna(master_df[col].median(), inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_1692\1060689562.py:199: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



✓ Saved: ../data/processed\flood_master.csv
  Rows: 8352
  Columns: 30
  Size: 2.19 MB

STEP 6: FILE AUDIT

Columns:
   1. Unnamed: 0
   2. State_Name
   3. DFSI
   4. Dist_Name
   5. Human_fatality
   6. Human_injured
   7. Population
   8. Mean_Flood_Duration
   9. UEI
  10. Start Date
  11. End Date
  12. Duration(Days)
  13. Main Cause
  14. Location
  15. Districts
  16. State
  17. Latitude
  18. Longitude
  19. Severity
  20. Area Affected
  21. Human fatality
  22. Human injured
  23. Human Displaced
  24. Animal Fatality
  25. Description of Casualties/injured
  26. Extent of damage 
  27. Event Source
  28. Event Souce ID
  29. District_LGD_Codes
  30. State_Codes

First row:
Unnamed: 0                               Patna
State_Name                               BIHAR
DFSI                                 19.300564
Dist_Name                                  NaN
Human_fatality                            50.0
Human_injured                              3.0
Population            